In [13]:
#import pymssql as sql
import pymysql as sql
import pandas as pd
from datetime import datetime, timedelta

In [2]:
class cook_object:

    def __init__(self):

        self.db_name = 'MICO_96'
        self.db_server = '10.2.4.146'
        self.db_login = 'asterisk'
        with open('sql.pass','r') as file:
            self.db_pass = file.read().replace('\n', '')
            file.close()
        self.con = self.connect_sql()

    def connect_sql(self):

        return sql.connect(
            self.db_server, 
            self.db_login, 
            self.db_pass,
            self.db_name
        )

In [3]:
cook = cook_object()

linkedid = []
linkedid.append(
    {
    'call_id':'0012388684',
    'date_from':'2020-10-15 22:15:06',
    'date_to':'2020-10-15 22:17:06',
    'linkedid':'1602789350.14400182',
    }
)
linkedid.append(
    {
    'call_id':'0012388685',
    'date_from':'2020-10-15 22:18:25',
    'date_to':'2020-10-15 22:20:25',
    'linkedid':'1602789550.14400185',
    }
)

query = 'create temporary table primary_linkedid_0'
i = 0
for instance in linkedid:
    query+="""
    """+('' if i==0 else 'union all')+"""
    select
    '"""+instance['call_id']+"""' as call_id,
    '"""+instance['date_from']+"""' as date_from,
    '"""+instance['date_to']+"""' as date_to,
    '"""+instance['linkedid']+"""' as linkedid"""
    i+=1
query+=';'
query

In [4]:
df = pd.read_csv('data/test.csv',';', dtype = {'call_id': 'str', 'linkedid': 'str'})
df

,call_id,date_from,date_to,linkedid
0,0012460591,27.10.2020 14:26:23,27.10.2020 14:28:23,1603798019.14676995


In [10]:
with cook.con:
    # drop temporary tables
    query = "drop table if exists primary_linkedid_0;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = "drop table if exists primary_linkedid_1;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = "drop table if exists primary_linkedid_2;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = "drop table if exists linkedid_relation_batch_1_instance_0;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = "drop table if exists linkedid_relation_batch_1_instance_1;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = "drop table if exists linkedid_relation_batch_1_instance_2;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = "drop table if exists linkedid_relation_batch_2_instance_0;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = "drop table if exists linkedid_related;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    # create known linkedid list
    query = 'create temporary table primary_linkedid_0'
    for i in range(len(df)):
        instance = df.iloc[i]
        date_from	= datetime.strptime(instance.date_from,	'%d.%m.%Y %H:%M:%S').strftime('%Y-%m-%dT%H:%M:%S')
        date_to		= datetime.strptime(instance.date_to,	'%d.%m.%Y %H:%M:%S').strftime('%Y-%m-%dT%H:%M:%S')
        query+="""
        """+('' if i==0 else 'union all')+"""
        select
        '"""+str(instance.call_id)+"""' as call_id,
        '"""+str(date_from)+"""' as date_from,
        '"""+str(date_to)+"""' as date_to,
        '"""+str(instance.linkedid)+"""' as linkedid"""

    query+=';'
    #print(query)
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    # duplicate
    query = """create temporary table primary_linkedid_1
    select
    call_id,
    date_from,
    date_to,
    linkedid
    from primary_linkedid_0;"""    
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = """create temporary table primary_linkedid_2
    select
    call_id,
    date_from,
    date_to,
    linkedid
    from primary_linkedid_0;"""
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    # union -> relation batch 1
    query = """create temporary table linkedid_relation_batch_1_instance_0
    select 
    primary_linkedid_0.call_id,
    primary_linkedid_0.date_from,
    primary_linkedid_0.date_to,
    primary_linkedid_0.linkedid,
    0 as relation_level
    from primary_linkedid_0 as primary_linkedid_0
    union all
    
    select
    primary_linkedid_1.call_id,
    primary_linkedid_1.date_from,
    primary_linkedid_1.date_to,
    relationsCalls.linkedid1,
    1
    from primary_linkedid_1 as primary_linkedid_1
    inner join relationsCalls 
    on 
    relationsCalls.calldate>primary_linkedid_1.date_from and
    relationsCalls.calldate<primary_linkedid_1.date_to and
    primary_linkedid_1.linkedid=relationsCalls.linkedid2
    union all
    
    select
    primary_linkedid_2.call_id,
    primary_linkedid_2.date_from,
    primary_linkedid_2.date_to,
    relationsCalls.linkedid2,
    1    
    from primary_linkedid_2 as primary_linkedid_2
    inner join relationsCalls 
    on 
    relationsCalls.calldate>primary_linkedid_2.date_from and
    relationsCalls.calldate<primary_linkedid_2.date_to and
    primary_linkedid_2.linkedid=relationsCalls.linkedid1;
    """
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    
    # duplicate relation batch 1
    query = """create temporary table linkedid_relation_batch_1_instance_1
    select 
    call_id,
    date_from,
    date_to,
    linkedid,
    relation_level
    from linkedid_relation_batch_1_instance_0;"""
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = """create temporary table linkedid_relation_batch_1_instance_2
    select 
    call_id,
    date_from,
    date_to,
    linkedid,
    relation_level
    from linkedid_relation_batch_1_instance_0;"""
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    # union -> relation batch 2
    query = """create temporary table linkedid_relation_batch_2_instance_0    
    select 
    linkedid_relation_batch_1_instance_0.call_id,
    linkedid_relation_batch_1_instance_0.date_from,
    linkedid_relation_batch_1_instance_0.date_to,
    linkedid_relation_batch_1_instance_0.linkedid
    from linkedid_relation_batch_1_instance_0 as linkedid_relation_batch_1_instance_0    
    union all
    
    select 
    linkedid_relation_batch_1_instance_1.call_id,
    linkedid_relation_batch_1_instance_1.date_from,
    linkedid_relation_batch_1_instance_1.date_to,
    relationsCalls.linkedid2
    from linkedid_relation_batch_1_instance_1 as linkedid_relation_batch_1_instance_1
    inner join relationsCalls 
    on 
    relationsCalls.calldate>linkedid_relation_batch_1_instance_1.date_from and
    relationsCalls.calldate<linkedid_relation_batch_1_instance_1.date_to and
    linkedid_relation_batch_1_instance_1.linkedid=relationsCalls.linkedid1
    where 
    linkedid_relation_batch_1_instance_1.relation_level = 1
    union all
    
    select 
    linkedid_relation_batch_1_instance_2.call_id,
    linkedid_relation_batch_1_instance_2.date_from,
    linkedid_relation_batch_1_instance_2.date_to,
    relationsCalls.linkedid1
    from linkedid_relation_batch_1_instance_2 as linkedid_relation_batch_1_instance_2
    inner join relationsCalls 
    on 
    relationsCalls.calldate>linkedid_relation_batch_1_instance_2.date_from and
    relationsCalls.calldate<linkedid_relation_batch_1_instance_2.date_to and
    linkedid_relation_batch_1_instance_2.linkedid=relationsCalls.linkedid2
    where 
    linkedid_relation_batch_1_instance_2.relation_level = 1;
    """
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    # group related linkedid
    query = """create temporary table linkedid_related
    select distinct    
    call_id,
    date_from,
    date_to,
    linkedid COLLATE utf8_unicode_ci as linkedid
    from linkedid_relation_batch_2_instance_0"""
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = """
    select
    linkedid_related.call_id,    
    linkedid_related.linkedid,
    PT1C_cdr_MICO.recordingfile
    from PT1C_cdr_MICO as PT1C_cdr_MICO
    inner join linkedid_related as linkedid_related on
    PT1C_cdr_MICO.calldate>linkedid_related.date_from and
    PT1C_cdr_MICO.calldate<linkedid_related.date_to and
    PT1C_cdr_MICO.linkedid = linkedid_related.linkedid"""
    
    cursor = cook.con.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print(row)

('0012460591', '1603798019.14676995', '2020-10/27/in_9651688123_2020-10-27-14-27-00.wav49')


In [59]:
with cook.con:        
    query = "SHOW COLUMNS FROM PT1C_cdr_MICO;"
    #query = "SHOW COLUMNS FROM relationsCalls;"
    
    cursor = cook.con.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print(row)

('id', 'int(11) unsigned', 'NO', 'PRI', None, 'auto_increment')
('calldate', 'datetime', 'NO', 'MUL', '0000-00-00 00:00:00', '')
('clid', 'varchar(80)', 'NO', 'MUL', '', '')
('src', 'varchar(80)', 'NO', 'MUL', '', '')
('dst', 'varchar(80)', 'NO', 'MUL', '', '')
('dcontext', 'varchar(80)', 'NO', 'MUL', '', '')
('lastapp', 'varchar(200)', 'NO', '', '', '')
('lastdata', 'varchar(200)', 'NO', '', '', '')
('duration', 'float unsigned', 'YES', '', None, '')
('billsec', 'float unsigned', 'YES', '', None, '')
('disposition', "enum('ANSWERED','BUSY','FAILED','NO ANSWER','CONGESTION')", 'YES', '', None, '')
('channel', 'varchar(50)', 'YES', '', None, '')
('dstchannel', 'varchar(50)', 'YES', '', None, '')
('amaflags', 'varchar(50)', 'YES', '', None, '')
('accountcode', 'varchar(20)', 'YES', '', None, '')
('uniqueid', 'varchar(32)', 'NO', '', '', '')
('userfield', 'varchar(200)', 'NO', '', '', '')
('answer', 'datetime', 'NO', '', None, '')
('end', 'datetime', 'NO', '', None, '')
('recordingfile', 

In [68]:
query = """
    select
    PT1C_cdr_MICO.id,
    PT1C_cdr_MICO.calldate,
    PT1C_cdr_MICO.recordingfile
    from PT1C_cdr_MICO as PT1C_cdr_MICO
    where PT1C_cdr_MICO.linkedid = '1603798019.14676995' limit 1;"""
    
cursor = cook.con.cursor()
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

(12859665, datetime.datetime(2020, 10, 27, 14, 26, 59), '2020-10/27/in_9651688123_2020-10-27-14-27-00.wav49')


In [56]:
with cook.con:        
    #query = "SHOW COLUMNS FROM PT1C_cdr_MICO;"
    query = """
    select
                        '12460597' as call_id,
                        '2020-10-26T14:27:56' as date_from,
                        '2020-10-28T14:27:56' as date_to,
                        '1603798063.1698697' as linkedid
                        union all
                        select
                        '12460596' as call_id,
                        '2020-10-26T14:27:55' as date_from,
                        '2020-10-28T14:27:55' as date_to,
                        '1603798062.1698697' as linkedid
                        union all
                        select
                        '12460595' as call_id,
                        '2020-10-26T14:27:45' as date_from,
                        '2020-10-28T14:27:45' as date_to,
                        '1603798052.1698694' as linkedid;
    """
    
    cursor = cook.con.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print(row)

('12460597', '2020-10-26T14:27:56', '2020-10-28T14:27:56', '1603798063.1698697')
('12460596', '2020-10-26T14:27:55', '2020-10-28T14:27:55', '1603798062.1698697')
('12460595', '2020-10-26T14:27:45', '2020-10-28T14:27:45', '1603798052.1698694')


### Get linkedid by filename

In [40]:
def linkedid_by_filename(filename,y,m,d):
    filename = filename.replace('rxtx.wav','')
    date_from = datetime(int(y),int(m),int(d))
    date_toto = date_from+timedelta(days=1)
    date_from = datetime.strptime(str(date_from), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%dT%H:%M:%S')
    date_toto = datetime.strptime(str(date_toto), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%dT%H:%M:%S')
    with cook.con:
        query = """
        select
            linkedid
            from PT1C_cdr_MICO as PT1C_cdr_MICO
            where 
                calldate>'"""+date_from+"""' and 
                calldate<'"""+date_toto+"""' and 
                PT1C_cdr_MICO.recordingfile LIKE '%"""+filename+"""%' 
                limit 1
        """

        cursor = cook.con.cursor()
        cursor.execute(query)
        for row in cursor.fetchall():
            return row[0]
    return ''

In [41]:
linkedid_by_filename('in_78312536256_2020-11-19-09-06-14rxtx.wav','2020', '11', '19')

'1605765973.15209945'